In [1]:
import numpy as np
import pandas as pd
import os
import math

from scipy import stats #Analysis 
from scipy.stats import norm 

import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline

In [2]:
path = "input/"

# original data : data 
data =  pd.read_csv(path + "demential_data.csv", parse_dates=['summary_date']) # 애초에 datatime 유형의 데이터를 읽어 올 때부터 형변환하기
data

,EMAIL,summary_date,activity_average_met,activity_cal_active,activity_cal_total,activity_class_5min,activity_daily_movement,activity_high,activity_inactive,activity_inactivity_alerts,...,sleep_temperature_deviation,sleep_temperature_trend_deviation,timezone,sleep_total,CONVERT(activity_class_5min USING utf8),CONVERT(activity_met_1min USING utf8),CONVERT(sleep_hr_5min USING utf8),CONVERT(sleep_hypnogram_5min USING utf8),CONVERT(sleep_rmssd_5min USING utf8),DIAG_NM
0,nia+404@rowan.kr,2020-11-27,1.71875,730,2944,...,14346,0,417,0,...,-0.12,99.99,NaN,\r,2/1/1/1/1/1/2/2/1/1/1/1/1/1/2/2/2/3/2/2/2/2/2/...,0.9/0.9/1.4/1.9/1.1/0.9/0.9/1.1/1.3/1/0.9/1.1/...,0/73/73/73/72/71/70/71/71/71/70/70/73/72/74/74...,4/2/4/3/3/1/2/2/2/2/2/2/3/3/3/4/4/3/2/2/2/2/2/...,0/10/10/10/11/11/10/12/18/13/14/12/10/10/18/17...,CN
1,nia+404@rowan.kr,2020-11-28,1.40625,342,2449,...,6352,0,473,0,...,-0.32,99.99,NaN,\r,1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/2/2/2/2/2/2/...,1.2/1.1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,69/70/69/69/70/72/71/72/70/69/69/69/68/68/63/6...,2/4/2/2/2/2/3/1/2/2/4/4/2/2/2/2/2/2/2/2/2/2/4/...,23/23/26/24/18/13/15/14/17/20/24/30/23/25/22/1...,CN
2,nia+404@rowan.kr,2020-11-29,1.46875,401,2544,...,7297,0,586,0,...,0.07,99.99,NaN,\r,1/1/1/1/1/1/1/2/1/1/1/1/2/2/2/2/2/1/1/1/1/1/2/...,1.1/1.1/1.2/1.1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,0/74/73/73/74/74/74/71/71/70/70/69/70/68/66/69...,4/2/4/4/1/1/1/4/4/4/4/4/4/4/2/3/4/2/2/4/2/2/2/...,0/11/14/20/13/14/14/16/27/29/27/20/19/19/14/12...,CN
3,nia+404@rowan.kr,2020-11-30,0.34375,27,1850,...,491,0,176,0,...,-0.41,99.99,NaN,\r,2/1/2/2/1/2/1/1/2/1/1/1/1/1/2/1/1/1/1/1/2/2/2/...,0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/...,73/70/71/72/75/75/73/70/70/70/67/63/63/63/63/6...,4/4/4/4/3/3/3/2/4/4/4/2/2/2/2/2/2/2/2/4/2/2/2/...,24/28/19/17/12/10/17/20/23/23/25/31/26/25/34/3...,CN
4,nia+404@rowan.kr,2020-12-01,1.46875,333,2518,...,5861,0,646,0,...,-0.27,99.99,NaN,\r,1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/2/2/3/3/2/...,0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,0/0/0/0/0/0/0/0/69/69/71/69/65/66/64/64/65/66/...,4/4/4/4/4/4/4/4/4/4/4/2/2/2/2/3/3/2/4/4/4/2/2/...,0/0/0/0/0/0/0/0/21/22/26/23/19/29/22/17/14/13/...,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,nia+206@rowan.kr,2020-12-22,1.34375,227,2316,...,3863,3,735,2,...,-0.16,99.99,NaN,\r,2/1/1/2/2/1/1/1/1/2/2/3/2/1/1/1/1/1/1/1/1/1/1/...,1.1/1.4/1.2/0.9/1.2/1.1/0.9/0.9/0.9/1/0.9/0.9/...,0/54/54/54/55/56/0/0/55/52/52/53/54/54/56/57/6...,4/4/4/2/2/2/2/4/2/2/2/1/1/1/1/4/2/2/3/3/3/3/3/...,0/35/39/28/26/41/0/0/52/31/27/30/21/22/27/40/3...,CN
9323,nia+206@rowan.kr,2020-12-24,1.34375,249,2351,...,4411,1,780,4,...,-0.09,99.99,NaN,\r,1/1/1/1/1/1/1/1/1/1/2/3/2/2/2/1/1/1/1/1/1/1/1/...,0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,68/66/67/67/68/69/69/70/71/71/71/69/72/70/70/7...,4/4/2/2/2/1/1/1/1/1/1/2/2/3/3/2/2/2/2/2/3/4/2/...,26/16/19/18/16/18/19/17/15/16/15/13/11/13/13/1...,CN
9324,nia+206@rowan.kr,2020-12-26,1.53125,570,2682,...,11057,1,518,1,...,0.41,99.99,NaN,\r,1/1/1/1/1/1/2/2/2/2/2/2/1/1/1/1/1/1/1/1/1/1/1/...,0.9/0.9/0.9/0.9/1.1/0.9/0.9/1.1/0.9/0.9/0.9/0....,0/0/65/65/66/67/69/72/73/72/73/74/73/74/75/75/...,4/4/2/2/2/2/2/2/2/2/2/2/4/4/4/4/2/3/3/3/2/2/4/...,0/0/13/12/12/13/12/12/12/16/13/12/13/17/13/18/...,CN
9325,nia+206@rowan.kr,2020-12-27,1.34375,295,2331,...,5135,4,578,0,...,0.27,99.99,NaN,\r,1/1/1/2/3/2/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/...,1.1/0.9/0.9/0.9/1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/...,0/0/0/64/64/64/64/65/65/66/66/66/66/68/68/68/6...,4/4/4/4/2/2/2/2/2/2/2/2/2/3/3/3/3/2/4/4/2/2/2/...,0/0/0/14/17/18/14/14/17/15/17/12/17/12/13/12/1...,CN


# Preprocessing

#### feature drop
- timezone
- EMAIL
- sleep_temperature_trend_deviation, sleep_is_longest, sleep_total
    - 모든 값이 1, '\r', 99.99 이므로 제거
- sleep_temperature_deviation
    - sleep_temperature_delta 변수와 중복되므로 deviation 변수를 제거
- activity_class_5min, activity_met_1min
    - convert data
- sleep_hr_5min, sleep_hypnogram_5min, sleep_rmssd_5min
    - convert data
- CONVERT(activity_class_5min USING utf8)
- CONVERT(activity_met_1min USING utf8)
- CONVERT(sleep_hr_5min USING utf8)
- CONVERT(sleep_hypnogram_5min USING utf8)
- CONVERT(sleep_rmssd_5min USING utf8)

#### y variable
- DIAG_NM
    - 정상(CN), 경도인지 장애(MCI), 치매(Dem)

In [3]:
# column의 수가 많기 때문에 효율적으로 관리하기 위해서 
# column의 이름들을 묶어서 진행합시다

drop_cols = ['timezone', 'EMAIL',
             'activity_class_5min', 'activity_met_1min', 
             'sleep_hr_5min', 'sleep_hypnogram_5min', 'sleep_rmssd_5min',
             'sleep_is_longest', 'sleep_temperature_trend_deviation', 'sleep_total', 'sleep_temperature_deviation']
convert_cols = data.iloc[:, 60:66].columns
data_convert = data[convert_cols].copy()
data_convert

,CONVERT(activity_class_5min USING utf8),CONVERT(activity_met_1min USING utf8),CONVERT(sleep_hr_5min USING utf8),CONVERT(sleep_hypnogram_5min USING utf8),CONVERT(sleep_rmssd_5min USING utf8),DIAG_NM
0,2/1/1/1/1/1/2/2/1/1/1/1/1/1/2/2/2/3/2/2/2/2/2/...,0.9/0.9/1.4/1.9/1.1/0.9/0.9/1.1/1.3/1/0.9/1.1/...,0/73/73/73/72/71/70/71/71/71/70/70/73/72/74/74...,4/2/4/3/3/1/2/2/2/2/2/2/3/3/3/4/4/3/2/2/2/2/2/...,0/10/10/10/11/11/10/12/18/13/14/12/10/10/18/17...,CN
1,1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/2/2/2/2/2/2/...,1.2/1.1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,69/70/69/69/70/72/71/72/70/69/69/69/68/68/63/6...,2/4/2/2/2/2/3/1/2/2/4/4/2/2/2/2/2/2/2/2/2/2/4/...,23/23/26/24/18/13/15/14/17/20/24/30/23/25/22/1...,CN
2,1/1/1/1/1/1/1/2/1/1/1/1/2/2/2/2/2/1/1/1/1/1/2/...,1.1/1.1/1.2/1.1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,0/74/73/73/74/74/74/71/71/70/70/69/70/68/66/69...,4/2/4/4/1/1/1/4/4/4/4/4/4/4/2/3/4/2/2/4/2/2/2/...,0/11/14/20/13/14/14/16/27/29/27/20/19/19/14/12...,CN
3,2/1/2/2/1/2/1/1/2/1/1/1/1/1/2/1/1/1/1/1/2/2/2/...,0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/...,73/70/71/72/75/75/73/70/70/70/67/63/63/63/63/6...,4/4/4/4/3/3/3/2/4/4/4/2/2/2/2/2/2/2/2/4/2/2/2/...,24/28/19/17/12/10/17/20/23/23/25/31/26/25/34/3...,CN
4,1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/2/2/3/3/2/...,0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,0/0/0/0/0/0/0/0/69/69/71/69/65/66/64/64/65/66/...,4/4/4/4/4/4/4/4/4/4/4/2/2/2/2/3/3/2/4/4/4/2/2/...,0/0/0/0/0/0/0/0/21/22/26/23/19/29/22/17/14/13/...,CN
...,...,...,...,...,...,...
9322,2/1/1/2/2/1/1/1/1/2/2/3/2/1/1/1/1/1/1/1/1/1/1/...,1.1/1.4/1.2/0.9/1.2/1.1/0.9/0.9/0.9/1/0.9/0.9/...,0/54/54/54/55/56/0/0/55/52/52/53/54/54/56/57/6...,4/4/4/2/2/2/2/4/2/2/2/1/1/1/1/4/2/2/3/3/3/3/3/...,0/35/39/28/26/41/0/0/52/31/27/30/21/22/27/40/3...,CN
9323,1/1/1/1/1/1/1/1/1/1/2/3/2/2/2/1/1/1/1/1/1/1/1/...,0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,68/66/67/67/68/69/69/70/71/71/71/69/72/70/70/7...,4/4/2/2/2/1/1/1/1/1/1/2/2/3/3/2/2/2/2/2/3/4/2/...,26/16/19/18/16/18/19/17/15/16/15/13/11/13/13/1...,CN
9324,1/1/1/1/1/1/2/2/2/2/2/2/1/1/1/1/1/1/1/1/1/1/1/...,0.9/0.9/0.9/0.9/1.1/0.9/0.9/1.1/0.9/0.9/0.9/0....,0/0/65/65/66/67/69/72/73/72/73/74/73/74/75/75/...,4/4/2/2/2/2/2/2/2/2/2/2/4/4/4/4/2/3/3/3/2/2/4/...,0/0/13/12/12/13/12/12/12/16/13/12/13/17/13/18/...,CN
9325,1/1/1/2/3/2/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/...,1.1/0.9/0.9/0.9/1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/...,0/0/0/64/64/64/64/65/65/66/66/66/66/68/68/68/6...,4/4/4/4/2/2/2/2/2/2/2/2/2/3/3/3/3/2/4/4/2/2/2/...,0/0/0/14/17/18/14/14/17/15/17/12/17/12/13/12/1...,CN


In [4]:
data_convert.iat[0, 2].count('/')

119

### data_convert '/' 제거, label Encoding

#### - CONVERT(activity_class_5min USING utf8)

In [5]:
signal_list = list(data_convert['CONVERT(activity_class_5min USING utf8)'].str.split('/'))
signal_activity_class_5min = pd.DataFrame(signal_list).drop(288, axis=1).astype('float')
y_df = data_convert['DIAG_NM'] 
signal_activity_class_5min = pd.concat([signal_activity_class_5min, y_df], axis=1)
signal_activity_class_5min

,0,1,2,3,4,5,6,7,8,9,...,279,280,281,282,283,284,285,286,287,DIAG_NM
0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,...,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN
3,2.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,2.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN
9323,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN
9324,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN
9325,1.0,1.0,1.0,2.0,3.0,2.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CN


#### - CONVERT(activity_met_1min USING utf8)

In [6]:
signal_list = list(data_convert['CONVERT(activity_met_1min USING utf8)'].str.split('/'))
signal_activity_met_1min = pd.DataFrame(signal_list).drop(1440, axis=1).astype('float')
signal_activity_met_1min = pd.concat([signal_activity_met_1min, y_df], axis=1)
signal_activity_met_1min

,0,1,2,3,4,5,6,7,8,9,...,1431,1432,1433,1434,1435,1436,1437,1438,1439,DIAG_NM
0,0.9,0.9,1.4,1.9,1.1,0.9,0.9,1.1,1.3,1.0,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN
1,1.2,1.1,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN
2,1.1,1.1,1.2,1.1,0.9,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
4,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,1.1,1.4,1.2,0.9,1.2,1.1,0.9,0.9,0.9,1.0,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN
9323,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN
9324,0.9,0.9,0.9,0.9,1.1,0.9,0.9,1.1,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN
9325,1.1,0.9,0.9,0.9,1.0,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,CN


#### - CONVERT(sleep_hr_5min USING utf8)

In [7]:
signal_list = list(data_convert['CONVERT(sleep_hr_5min USING utf8)'].str.split('/'))
signal_sleep_hr_5min = pd.DataFrame(signal_list)
signal_sleep_hr_5min

,0,1,2,3,4,5,6,7,8,9,...,172,173,174,175,176,177,178,179,180,181
0,0,73,73,73,72,71,70,71,71,71,...,None,None,None,None,None,None,None,None,None,None
1,69,70,69,69,70,72,71,72,70,69,...,None,None,None,None,None,None,None,None,None,None
2,0,74,73,73,74,74,74,71,71,70,...,None,None,None,None,None,None,None,None,None,None
3,73,70,71,72,75,75,73,70,70,70,...,None,None,None,None,None,None,None,None,None,None
4,0,0,0,0,0,0,0,0,69,69,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,0,54,54,54,55,56,0,0,55,52,...,None,None,None,None,None,None,None,None,None,None
9323,68,66,67,67,68,69,69,70,71,71,...,None,None,None,None,None,None,None,None,None,None
9324,0,0,65,65,66,67,69,72,73,72,...,None,None,None,None,None,None,None,None,None,None
9325,0,0,0,64,64,64,64,65,65,66,...,None,None,None,None,None,None,None,None,None,None


In [8]:
signal_sleep_hr_5min = signal_sleep_hr_5min.drop(181, axis=1).transpose()
# 결측치 처리 Nan, '', ' ' -> 우선 다 int type 0으로 replace 
signal_sleep_hr_5min = signal_sleep_hr_5min.fillna(0) 
signal_sleep_hr_5min = signal_sleep_hr_5min.replace('', 0)
signal_sleep_hr_5min = signal_sleep_hr_5min.replace(' ', 0)
signal_sleep_hr_5min = signal_sleep_hr_5min.astype('float')

In [9]:
signal_sleep_hr_5min  = signal_sleep_hr_5min.transpose()
signal_sleep_hr_5min = pd.concat([signal_sleep_hr_5min, y_df], axis=1)
signal_sleep_hr_5min

,0,1,2,3,4,5,6,7,8,9,...,172,173,174,175,176,177,178,179,180,DIAG_NM
0,0.0,73.0,73.0,73.0,72.0,71.0,70.0,71.0,71.0,71.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
1,69.0,70.0,69.0,69.0,70.0,72.0,71.0,72.0,70.0,69.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
2,0.0,74.0,73.0,73.0,74.0,74.0,74.0,71.0,71.0,70.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
3,73.0,70.0,71.0,72.0,75.0,75.0,73.0,70.0,70.0,70.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,69.0,69.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,0.0,54.0,54.0,54.0,55.0,56.0,0.0,0.0,55.0,52.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9323,68.0,66.0,67.0,67.0,68.0,69.0,69.0,70.0,71.0,71.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9324,0.0,0.0,65.0,65.0,66.0,67.0,69.0,72.0,73.0,72.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9325,0.0,0.0,0.0,64.0,64.0,64.0,64.0,65.0,65.0,66.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN


#### - CONVERT(sleep_hypnogram_5min USING utf8)

In [10]:
signal_list = list(data_convert['CONVERT(sleep_hypnogram_5min USING utf8)'].str.split('/'))
signal_sleep_hypnogram_5min = pd.DataFrame(signal_list)

signal_sleep_hypnogram_5min = signal_sleep_hypnogram_5min.drop(180, axis=1).transpose()
# 결측치 처리 Nan, '', ' ' -> 우선 다 int type 0으로 replace 
signal_sleep_hypnogram_5min = signal_sleep_hypnogram_5min.fillna(0) 
signal_sleep_hypnogram_5min = signal_sleep_hypnogram_5min.replace('', 0)
signal_sleep_hypnogram_5min = signal_sleep_hypnogram_5min.replace(' ', 0)
signal_sleep_hypnogram_5min = signal_sleep_hypnogram_5min.astype('float')

In [11]:
signal_sleep_hypnogram_5min  = signal_sleep_hypnogram_5min.transpose()
signal_sleep_hypnogram_5min = pd.concat([signal_sleep_hypnogram_5min, y_df], axis=1)
signal_sleep_hypnogram_5min

,0,1,2,3,4,5,6,7,8,9,...,171,172,173,174,175,176,177,178,179,DIAG_NM
0,4.0,2.0,4.0,3.0,3.0,1.0,2.0,2.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
1,2.0,4.0,2.0,2.0,2.0,2.0,3.0,1.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
2,4.0,2.0,4.0,4.0,1.0,1.0,1.0,4.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
3,4.0,4.0,4.0,4.0,3.0,3.0,3.0,2.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
4,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,4.0,4.0,4.0,2.0,2.0,2.0,2.0,4.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9323,4.0,4.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9324,4.0,4.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9325,4.0,4.0,4.0,4.0,2.0,2.0,2.0,2.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN


#### - CONVERT(sleep_rmssd_5min USING utf8)

In [12]:
signal_list = list(data_convert['CONVERT(sleep_rmssd_5min USING utf8)'].str.split('/'))
signal_sleep_rmssd_5min = pd.DataFrame(signal_list)

signal_sleep_rmssd_5min = signal_sleep_rmssd_5min.drop(181, axis=1).transpose()
# 결측치 처리 Nan, '', ' ' -> 우선 다 int type 0으로 replace 
signal_sleep_rmssd_5min = signal_sleep_rmssd_5min.fillna(0) 
signal_sleep_rmssd_5min = signal_sleep_rmssd_5min.replace('', 0)
signal_sleep_rmssd_5min = signal_sleep_rmssd_5min.replace(' ', 0)
signal_sleep_rmssd_5min = signal_sleep_rmssd_5min.astype('float')

In [13]:
signal_sleep_rmssd_5min  = signal_sleep_rmssd_5min.transpose()
signal_sleep_rmssd_5min = pd.concat([signal_sleep_rmssd_5min, y_df], axis=1)
signal_sleep_rmssd_5min

,0,1,2,3,4,5,6,7,8,9,...,172,173,174,175,176,177,178,179,180,DIAG_NM
0,0.0,10.0,10.0,10.0,11.0,11.0,10.0,12.0,18.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
1,23.0,23.0,26.0,24.0,18.0,13.0,15.0,14.0,17.0,20.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
2,0.0,11.0,14.0,20.0,13.0,14.0,14.0,16.0,27.0,29.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
3,24.0,28.0,19.0,17.0,12.0,10.0,17.0,20.0,23.0,23.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,22.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322,0.0,35.0,39.0,28.0,26.0,41.0,0.0,0.0,52.0,31.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9323,26.0,16.0,19.0,18.0,16.0,18.0,19.0,17.0,15.0,16.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9324,0.0,0.0,13.0,12.0,12.0,13.0,12.0,12.0,12.0,16.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN
9325,0.0,0.0,0.0,14.0,17.0,18.0,14.0,14.0,17.0,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CN


In [14]:
signal1 = signal_activity_class_5min.values.tolist()
signal2 = signal_activity_met_1min.values.tolist()
signal3 = signal_sleep_hr_5min.values.tolist()
signal4 = signal_sleep_hypnogram_5min.values.tolist()
signal5 = signal_sleep_rmssd_5min.values.tolist()

In [15]:
temp = input_data_X1 = signal_activity_class_5min.drop(labels = 'DIAG_NM', axis = 1)


# Data Split

In [16]:
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold, GridSearchCV

# y 변수와 X 변수 분할 관리 
input_data_y = y_df.copy()
input_data_X1 = signal_activity_class_5min.drop(labels = 'DIAG_NM', axis = 1)
input_data_X2 = signal_activity_met_1min.drop(labels = 'DIAG_NM', axis = 1)
input_data_X3 = signal_sleep_hr_5min.drop(labels = 'DIAG_NM', axis = 1)
input_data_X4 = signal_sleep_hypnogram_5min.drop(labels = 'DIAG_NM', axis = 1)
input_data_X5 = signal_sleep_rmssd_5min.drop(labels = 'DIAG_NM', axis = 1)

In [17]:
# LabelEncoder
# y data를 LabelEncdoer로 한 번 더 labeling 합니다 -> 100만원, 10000만원... = 0, 1, ... 
# 추후에 DL에서 output을 맞춰주기 위함입니다, 최종 마지막에서 원래 label값(100만원, 1000만원...)으로 되돌립니다
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
input_data_y = le.fit_transform(list(input_data_y))

print("original gain_label \n", le.classes_)
print("gain_lable label \n", set(input_data_y))

original gain_label 
 ['CN' 'Dem' 'MCI']
gain_lable label 
 {0, 1, 2}


In [18]:
X1_train, X1_test, y1_train, y1_test = train_test_split(input_data_X1, input_data_y, test_size=0.2, shuffle=True, stratify=input_data_y, random_state=34)

In [19]:
X2_train, X2_test, y2_train, y2_test = train_test_split(input_data_X2, input_data_y, test_size=0.2, shuffle=True, stratify=input_data_y, random_state=34)
X3_train, X3_test, y3_train, y3_test = train_test_split(input_data_X3, input_data_y, test_size=0.2, shuffle=True, stratify=input_data_y, random_state=34)
X4_train, X4_test, y4_train, y4_test = train_test_split(input_data_X4, input_data_y, test_size=0.2, shuffle=True, stratify=input_data_y, random_state=34)
X5_train, X5_test, y5_train, y5_test = train_test_split(input_data_X5, input_data_y, test_size=0.2, shuffle=True, stratify=input_data_y, random_state=34)

In [20]:
print('훈련용  activity_class_5min: {}'.format(len(X1_train)))
print('테스트용 activity_class_5min : {}'.format(len(X1_test)))
num_classes = len(set(y1_train))
print('label: {}'.format(num_classes))

훈련용  activity_class_5min: 7461
테스트용 activity_class_5min : 1866
label: 3


In [21]:
unique_elements, counts_elements = np.unique(y1_train, return_counts=True)
print("activity_class_5min train에서 각 레이블에 대한 빈도수:")
print(np.asarray((unique_elements, counts_elements)))
unique_elements, counts_elements = np.unique(y1_test, return_counts=True)
print("activity_class_5min test에서 각 레이블에 대한 빈도수:")
print(np.asarray((unique_elements, counts_elements)))

activity_class_5min train에서 각 레이블에 대한 빈도수:
[[   0    1    2]
 [4775  513 2173]]
activity_class_5min test에서 각 레이블에 대한 빈도수:
[[   0    1    2]
 [1194  129  543]]


# Modeling - RNN

In [22]:
print("X3 :" ,X3_train.shape)
print("y3 :" ,y3_train.shape)
print("X4 :" ,X4_train.shape)
print("y4 :" ,y4_train.shape)
print("X5 :" ,X5_train.shape)
print("y5 :" ,y5_train.shape)

X3 : (7461, 181)
y3 : (7461,)
X4 : (7461, 180)
y4 : (7461,)
X5 : (7461, 181)
y5 : (7461,)


In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Activation, SimpleRNN
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from keras.layers import Bidirectional
from keras import optimizers
from sklearn.metrics import accuracy_score
from keras.layers import Bidirectional
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping

import tensorflow as tf

### 기본 RNN 모델

In [24]:
def vanilla_rnn():
    model = Sequential()
    model.add(SimpleRNN(20, input_shape = (181,1), return_sequences = False)) #input shape에 열 개수 넣어주면 됨
    model.add(Dense(3)) #target 개수 = 3
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss =  'sparse_categorical_crossentropy', optimizer = adam, metrics = ['accuracy']) #label encoding 했기때문에  'sparse_categorical_crossentropy' 사용
    
    return model

In [25]:
model = KerasClassifier(build_fn = vanilla_rnn, epochs = 10, batch_size = 256, verbose = 1)
model.fit(X3_train, y3_train)
y3_pred = model.predict(X3_test)
print("accouracy : ", accuracy_score(y3_pred, y3_test))

Epoch 1/10


<ipython-input-25-c421baef7525>:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn = vanilla_rnn, epochs = 10, batch_size = 256, verbose = 1)
C:\pythontemp\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


30/30 [==============================] - 1s 13ms/step - loss: 0.9580 - accuracy: 0.6202
Epoch 2/10
30/30 [==============================] - 0s 13ms/step - loss: 0.8540 - accuracy: 0.6372
Epoch 3/10
30/30 [==============================] - 0s 12ms/step - loss: 0.8317 - accuracy: 0.6380
Epoch 4/10
30/30 [==============================] - 0s 13ms/step - loss: 0.8291 - accuracy: 0.6377
Epoch 5/10
30/30 [==============================] - 0s 13ms/step - loss: 0.8286 - accuracy: 0.6384
Epoch 6/10
30/30 [==============================] - 0s 13ms/step - loss: 0.8279 - accuracy: 0.6387
Epoch 7/10
30/30 [==============================] - 0s 13ms/step - loss: 0.8282 - accuracy: 0.6383
Epoch 8/10
30/30 [==============================] - 0s 12ms/step - loss: 0.8271 - accuracy: 0.6387
Epoch 9/10
30/30 [==============================] - 0s 12ms/step - loss: 0.8272 - accuracy: 0.6389
Epoch 10/10
59/59 [==============================] - 0s 3ms/step
accouracy :  0.6414790996784566


### 다층 RNN 모델

In [26]:
def stacked_vanilla_rnn():
    model = Sequential()
    model.add(SimpleRNN(50, input_shape = (181,1), return_sequences = True))   # return_sequences parameter has to be set True to stack
    model.add(SimpleRNN(50, return_sequences = False))
    model.add(Dense(3))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [27]:
model = KerasClassifier(build_fn = stacked_vanilla_rnn, epochs = 10, batch_size = 256, verbose = 1)
model.fit(X3_train, y3_train)
y3_pred = model.predict(X3_test)
print("accouracy : ",accuracy_score(y3_pred, y3_test))

Epoch 1/10


<ipython-input-27-acd879203ece>:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn = stacked_vanilla_rnn, epochs = 10, batch_size = 256, verbose = 1)
C:\pythontemp\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


30/30 [==============================] - 3s 65ms/step - loss: 0.8466 - accuracy: 0.6253
Epoch 2/10
30/30 [==============================] - 2s 66ms/step - loss: 0.8263 - accuracy: 0.6387
Epoch 3/10
30/30 [==============================] - 2s 64ms/step - loss: 0.8257 - accuracy: 0.6401
Epoch 4/10
30/30 [==============================] - 2s 64ms/step - loss: 0.8215 - accuracy: 0.6395
Epoch 5/10
30/30 [==============================] - 2s 65ms/step - loss: 0.8281 - accuracy: 0.6401
Epoch 6/10
30/30 [==============================] - 2s 65ms/step - loss: 0.8227 - accuracy: 0.6405
Epoch 7/10
30/30 [==============================] - 2s 65ms/step - loss: 0.8290 - accuracy: 0.6369
Epoch 8/10
30/30 [==============================] - 2s 66ms/step - loss: 0.8246 - accuracy: 0.6405
Epoch 9/10
30/30 [==============================] - 2s 66ms/step - loss: 0.8208 - accuracy: 0.6404
Epoch 10/10
59/59 [==============================] - 1s 8ms/step
accouracy :  0.6387995712754555


### 단층 LSTM 모델

In [28]:
def lstm():
    model = Sequential()
    model.add(LSTM(200, input_shape = (181,1), return_sequences = False))
    model.add(Dense(3))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [29]:
model = KerasClassifier(build_fn = lstm, epochs = 10, batch_size = 256, verbose = 1)
model.fit(X3_train, y3_train)
y3_pred = model.predict(X3_test)
print("accuracy : " , accuracy_score(y3_pred, y3_test))

<ipython-input-29-3a8aaed6cd0c>:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn = lstm, epochs = 10, batch_size = 256, verbose = 1)
C:\pythontemp\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/10
30/30 [==============================] - 13s 398ms/step - loss: 0.8942 - accuracy: 0.6309
Epoch 2/10
30/30 [==============================] - 12s 396ms/step - loss: 0.8268 - accuracy: 0.6399
Epoch 3/10
30/30 [==============================] - 12s 390ms/step - loss: 0.8248 - accuracy: 0.6395
Epoch 4/10
30/30 [==============================] - 12s 408ms/step - loss: 0.8175 - accuracy: 0.6396
Epoch 5/10
30/30 [==============================] - 12s 414ms/step - loss: 0.8134 - accuracy: 0.6400
Epoch 6/10
30/30 [==============================] - 13s 418ms/step - loss: 0.8100 - accuracy: 0.6400
Epoch 7/10
30/30 [==============================] - 13s 428ms/step - loss: 0.8110 - accuracy: 0.6400
Epoch 8/10
30/30 [==============================] - 13s 432ms/step - loss: 0.8105 - accuracy: 0.6401
Epoch 9/10
30/30 [==============================] - 13s 426ms/step - loss: 0.8090 - accuracy: 0.6403
Epoch 10/10
59/59 [==============================] - 2s 34ms/step
accuracy :  0.63987138263

### 다층 LSTM 모델

In [30]:
def stacked_lstm():
    model = Sequential()
    model.add(LSTM(20, input_shape = (181,1), return_sequences = True))
    model.add(LSTM(20, return_sequences = False))
    model.add(Dense(3))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [31]:
model = KerasClassifier(build_fn = stacked_lstm, epochs = 10, batch_size = 256, verbose = 1)
model.fit(X3_train, y3_train)
y3_pred = model.predict(X3_test)
print("accuracy : " , accuracy_score(y3_pred, y3_test))

<ipython-input-31-4e234d696d29>:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn = stacked_lstm, epochs = 10, batch_size = 256, verbose = 1)


Epoch 1/10


C:\pythontemp\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


30/30 [==============================] - 6s 147ms/step - loss: 0.9938 - accuracy: 0.6269
Epoch 2/10
30/30 [==============================] - 4s 149ms/step - loss: 0.8232 - accuracy: 0.6400
Epoch 3/10
30/30 [==============================] - 4s 146ms/step - loss: 0.8182 - accuracy: 0.6397
Epoch 4/10
30/30 [==============================] - 4s 147ms/step - loss: 0.8135 - accuracy: 0.6380
Epoch 5/10
30/30 [==============================] - 4s 150ms/step - loss: 0.8097 - accuracy: 0.6376
Epoch 6/10
30/30 [==============================] - 4s 144ms/step - loss: 0.8093 - accuracy: 0.6384
Epoch 7/10
30/30 [==============================] - 4s 146ms/step - loss: 0.8099 - accuracy: 0.6400
Epoch 8/10
30/30 [==============================] - 4s 146ms/step - loss: 0.8078 - accuracy: 0.6399
Epoch 9/10
30/30 [==============================] - 4s 147ms/step - loss: 0.8078 - accuracy: 0.6399
Epoch 10/10
59/59 [==============================] - 1s 15ms/step
accuracy :  0.639871382636656


### 양방향 단층 LSTM 모델

In [32]:
def bidirectional_lstm():
    model = Sequential()
    model.add(Bidirectional(LSTM(20, return_sequences = False), input_shape = (181,1))) #input shape에 열 개수 넣어주면 됨
    model.add(Dense(3)) #target 개수 = 3
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

model = KerasClassifier(build_fn = bidirectional_lstm, epochs = 10, batch_size = 128, verbose = 1)
model.fit(X3_train, y3_train)
y3_pred = model.predict(X3_test)
print("accouracy : ",accuracy_score(y3_pred, y3_test))

<ipython-input-32-ac8a0de2f6ea>:12: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn = bidirectional_lstm, epochs = 10, batch_size = 128, verbose = 1)


Epoch 1/10


C:\pythontemp\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


59/59 [==============================] - 5s 41ms/step - loss: 0.9061 - accuracy: 0.6387
Epoch 2/10
59/59 [==============================] - 3s 44ms/step - loss: 0.8117 - accuracy: 0.6401
Epoch 3/10
59/59 [==============================] - 3s 44ms/step - loss: 0.8072 - accuracy: 0.6401
Epoch 4/10
59/59 [==============================] - 3s 43ms/step - loss: 0.8070 - accuracy: 0.6400
Epoch 5/10
59/59 [==============================] - 3s 44ms/step - loss: 0.8060 - accuracy: 0.6400
Epoch 6/10
59/59 [==============================] - 3s 43ms/step - loss: 0.8043 - accuracy: 0.6401
Epoch 7/10
59/59 [==============================] - 3s 42ms/step - loss: 0.8030 - accuracy: 0.6400
Epoch 8/10
59/59 [==============================] - 3s 43ms/step - loss: 0.8014 - accuracy: 0.6404
Epoch 9/10
59/59 [==============================] - 3s 43ms/step - loss: 0.8010 - accuracy: 0.6407
Epoch 10/10
59/59 [==============================] - 1s 9ms/step
accouracy :  0.639871382636656


### 양뱡향 다층 LSTM 모델

In [33]:
# 다층 LSTM을 구현하기 위한 함수
def stacked_bidirectional_lstm():
    model = Sequential()
    model.add(Bidirectional(LSTM(20, return_sequences = True), input_shape = (181,1))) #input shape에 열 개수 넣어주면 됨
    model.add(Bidirectional(LSTM(20, return_sequences = True)))
    model.add(Bidirectional(LSTM(20, return_sequences = False)))
    model.add(Dense(3)) #target 개수 = 3
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [34]:
model = KerasClassifier(build_fn = stacked_bidirectional_lstm, epochs = 10, batch_size = 256, verbose = 1)
model.fit(X3_train, y3_train)
y3_pred = model.predict(X3_test)
print("accouracy : ",accuracy_score(y3_pred, y3_test))

<ipython-input-34-d3bc3a32440d>:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn = stacked_bidirectional_lstm, epochs = 10, batch_size = 256, verbose = 1)


Epoch 1/10


C:\pythontemp\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


30/30 [==============================] - 61s 2s/step - loss: 0.8773 - accuracy: 0.6388
Epoch 2/10
30/30 [==============================] - 56s 2s/step - loss: 0.8211 - accuracy: 0.6397
Epoch 3/10
30/30 [==============================] - 58s 2s/step - loss: 0.8100 - accuracy: 0.6397
Epoch 4/10
30/30 [==============================] - 57s 2s/step - loss: 0.8081 - accuracy: 0.6399
Epoch 5/10
30/30 [==============================] - 56s 2s/step - loss: 0.8070 - accuracy: 0.6401
Epoch 6/10
30/30 [==============================] - 57s 2s/step - loss: 0.8073 - accuracy: 0.6403
Epoch 7/10
30/30 [==============================] - 58s 2s/step - loss: 0.8066 - accuracy: 0.6400
Epoch 8/10
30/30 [==============================] - 57s 2s/step - loss: 0.8023 - accuracy: 0.6396
Epoch 9/10
30/30 [==============================] - 58s 2s/step - loss: 0.7999 - accuracy: 0.6400
Epoch 10/10
59/59 [==============================] - 3s 27ms/step
accouracy :  0.6420150053590568
